In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2

In [ ]:
%pip install split_folders
import splitfolders
input_folder = "../input/plantvillage-dataset/color"
output = "./output"

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

In [ ]:
import cv2

img1 = cv2.imread('./output/train/Blueberry___healthy/e623ffc6-6ad2-43b4-84df-bd3a8ca46a14___RS_HL 0412.JPG')
print(img1.shape)

plt.figure()
plt.imshow(img1) 
plt.show()


In [ ]:
datagen = ImageDataGenerator()
train_generator = datagen.flow_from_directory(
                  directory='./output/train/',
                  target_size=(256, 256), # resize to this size
                  color_mode="rgb", # for coloured images
                  batch_size=1, # number of images to extract from folder for every batch
                  class_mode="categorical", # classes to predict
                  seed=2021 # to make the result reproducible
                  )
validation_generator = datagen.flow_from_directory(
                  directory='./output/val/',
                  target_size=(256, 256), # resize to this size
                  color_mode="rgb", # for coloured images
                  batch_size=1, # number of images to extract from folder for every batch
                  class_mode="categorical", # classes to predict
                  seed=2021 # to make the result reproducible
                  )
test_generator = datagen.flow_from_directory(
                  directory='./output/test/',
                  target_size=(256, 256), # resize to this size
                  color_mode="rgb", # for coloured images
                  batch_size=1, # number of images to extract from folder for every batch
                  class_mode="categorical", # classes to predict
                  seed=2021 # to make the result reproducible
                  )

In [ ]:
import tensorflow as tf
from tensorflow import keras

base_model = keras.applications.Xception(weights='imagenet', input_shape=(256, 256, 3),include_top=False)
base_model.trainable = True
inputs = keras.Input(shape=(256, 256, 3))
x = base_model(inputs, training=True)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(38)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [46]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras

# base_model = keras.applications.Xception(weights='imagenet', input_shape=(256, 256, 3),include_top=False)
# base_model.trainable = True
# inputs = keras.Input(shape=(256, 256, 3))
# x = base_model(inputs, training=True)
# x = keras.layers.GlobalAveragePooling2D()(x)
# outputs = keras.layers.Dense(38,activation='softmax')(x)
# model = keras.Model(inputs, outputs)

# model.compile(optimizer=keras.optimizers.Adam(),
#               loss=keras.losses.CategoricalCrossentropy(),
#               metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
BATCH_SIZE = 32
n_epochs = 5
model_history = model.fit(train_generator,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validation_generator)

In [ ]:
model.save("my_Xception_model_total_softmax.h5")

In [7]:
from keras.models import load_model
 
# load model
model = load_model('../input/plant-village-xception/my_Xception_model.h5')
# summarize model.
model.summary()

In [18]:
x = model.evaluate(test_generator)

In [20]:
print(x)

In [21]:
pred = model.predict(test_generator)

In [25]:
predicted_class_indices = np.argmax(pred, axis=1)
print(predicted_class_indices)

In [31]:
labels = test_generator.class_indices
print(labels)

In [27]:
print(len(test_generator), len(predicted_class_indices))

In [29]:
test_img = os.listdir('./output/test/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/')
print(test_img)

In [41]:
from tensorflow.keras.preprocessing import image
img = image.load_img('./output/test/Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot/75d924d6-a630-4e26-9e74-5b1e6dfc5e22___RS_GLSp 4475.JPG')
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)
print(img_batch.shape)
p = model.predict(img_batch)
# print(p)
print(label,np.argmax(p, axis=1))